In [51]:
import numpy as np
%load_ext autoreload
%autoreload 2
from src.iduedu.modules.downloaders import get_routes_by_terr_name
from src.iduedu.modules.routes_parser import estimate_crs_for_overpass
from src.iduedu.modules.graph_builder import get_single_public_transport_graph
import pandas as pd

g = get_single_public_transport_graph('bus', osm_id=2369946)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

connect_points = [point for coords in connected_ways for point in (coords[0], coords[-1])]
distances = cdist(connect_points, connect_points)
n = distances.shape[0]
mask = (np.arange(n)[:, None] // 2) == (np.arange(n) // 2)
distances[mask] = np.inf

indexes = []


def relative_point(point: int):
    rel_point = point // 2 * 2
    return rel_point if rel_point != point else rel_point + 1


lines = []

connected_line = []
debug = []
first_con_1, first_con_2 = np.unravel_index(np.argmin(distances), distances.shape)
distances[first_con_1, :] = np.inf
distances[first_con_2, :] = np.inf
distances[:, first_con_2] = np.inf
distances[:, first_con_1] = np.inf
first_con_1_rel, first_con_2_rel = [relative_point(x) for x in (first_con_1, first_con_2)]
distances[first_con_1_rel, first_con_2_rel] = np.inf
distances[first_con_2_rel, first_con_1_rel] = np.inf
indexes.append((first_con_1, first_con_2))

line1, line2 = first_con_1 // 2, first_con_2 // 2

lines.append(line1)
lines.append(line2)
if first_con_1 % 2 == 1:
    connected_line += connected_ways[line1]
    debug.append(connected_ways[line1])
else:
    connected_line += connected_ways[line1][::-1]
    debug.append(connected_ways[line1][::-1])

if first_con_2 % 2 == 0:
    connected_line += connected_ways[line2]
    debug.append(connected_ways[line2])
else:
    connected_line += connected_ways[line2][::-1]
    debug.append(connected_ways[line2][::-1])

extreme_points = [first_con_1_rel, first_con_2_rel]

print(f'extrem_from_start {extreme_points}')

for i in range(len(connected_ways) - 2):
    position, ind = np.unravel_index(np.argmin(distances[extreme_points]), (2, n))
    next_con = (extreme_points[position], ind)
    rel_point = relative_point(next_con[1])
    distances[:, rel_point] = np.inf
    line = ind // 2
    lines.append(line)
    if position == 0:
        connected_line = connected_ways[line] + connected_line if ind % 2 == 1 else (
                connected_ways[line][::-1] + connected_line)
        debug.insert(0, connected_ways[line]) if ind % 2 == 1 else debug.insert(0, connected_ways[line][::-1])
        indexes.insert(0, next_con[::-1])
        extreme_points = [rel_point, extreme_points[1]]
    else:
        connected_line = connected_line + connected_ways[line] if ind % 2 == 0 else (
                connected_line + connected_ways[line][::-1])
        debug.append(connected_ways[line]) if ind % 2 == 1 else debug.append(connected_ways[line][::-1])
        indexes.append(next_con)
        extreme_points = [extreme_points[0], rel_point]

    distances[next_con[0], :] = np.inf
    distances[next_con[1], :] = np.inf
    distances[:, next_con[0]] = np.inf
    distances[:, next_con[1]] = np.inf
    rel_point_2 = relative_point(next_con[0])
    distances[rel_point, rel_point_2] = np.inf
    distances[rel_point_2, rel_point] = np.inf
    print('---------------')
    print(f'index: {i}')
    print(indexes)
    print(f'extreme_points {extreme_points}')
    print(f'added con {next_con}')


extrem_from_start [2, 5]
---------------
index: 0
[(1, 2), (3, 4)]
extreme_points [0, 5]
added con (2, 1)
---------------
index: 1
[(1, 2), (3, 4), (5, 7)]
extreme_points [0, 6]
added con (5, 7)


In [ ]:
connected_ways

In [ ]:
from shapely import LineString
import geopandas as gpd

gpd.GeoDataFrame(geometry=[LineString(new_connected_ways)], crs=32635).explore(tiles='CartoDB positron')

In [ ]:
t = [1, 2]
[(x // 2) * 2 if ((x // 2) * 2 != x) else ((x // 2) * 2 + 1) for x in t]

In [ ]:
%load_ext autoreload
%autoreload 2
from iduedu.modules.graph_builder import get_all_public_transport_graph

res = get_all_public_transport_graph(osm_id=3767134)

In [ ]:
import networkx as nx

nx.write_gml(res, 'spb+lo_iduedu.gml')

In [48]:
from shapely import from_wkt
import osmnx as ox
import geopandas as gpd
import networkx as nx


def nx_to_gdf(
        graph: nx.MultiDiGraph, nodes: bool = False, edges: bool = False, node_geometry=False, fill_edge_geometry=False
) -> gpd.GeoDataFrame | tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    for _, _, data in graph.edges(data=True):
        if "geometry" in data:
            data["geometry"] = from_wkt(str(data["geometry"]))

    if edges and nodes:
        gdf_graph_nodes, gdf_graph_edges = ox.graph_to_gdfs(
            graph, nodes=True, edges=True, node_geometry=node_geometry, fill_edge_geometry=fill_edge_geometry
        )
        gdf_graph_edges = gdf_graph_edges.reset_index()
        return gdf_graph_nodes, gdf_graph_edges
    elif edges:
        gdf_graph_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True, fill_edge_geometry=fill_edge_geometry)
        gdf_graph_edges = gdf_graph_edges.reset_index()
        return gdf_graph_edges
    elif nodes:
        gdf_graph_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False, node_geometry=node_geometry)
        return gdf_graph_nodes
    raise ValueError("You must specify either nodes or edges as True.")


g.graph['crs'] = 32638
nodes, edges = nx_to_gdf(nx.MultiDiGraph(g), nodes=True, edges=True, node_geometry=True)

In [49]:
m1 = edges.explore(column='route', tiles='CartoDB positron')
# nodes.explore(m=m1)
m1

In [ ]:
t = data_geom.iloc[3]
platforms = gpd.GeoDataFrame(geometry=t['platforms'], crs=local_crs)
stops = gpd.GeoDataFrame(geometry=t['stops'], crs=local_crs)

connection = gpd.sjoin_nearest(platforms, stops)
connection

In [ ]:
from shapely.ops import substring

line = LineString([(0, 0), (2, 2), (5, 5), (10, 0)])

start_distance = 4
end_distance = 6

sub_line = substring(line, 4, 10)
line

In [ ]:
m1 = gpd.GeoDataFrame(geometry=[LineString(x) for x in data_geom['path'].iloc[2:3]], crs=local_crs).explore()
gpd.GeoDataFrame(geometry=graph_gdf.iloc[2], crs=local_crs).explore(m=m1, color='red')

In [ ]:
from shapely import LineString
import geopandas as gpd

m1 = gpd.GeoDataFrame(geometry=[LineString(x) for x in data_geom['path']], crs=local_crs).reset_index().explore(
    column='index', tiles='CartoDB positron')
# gpd.GeoDataFrame(data_geom.iloc[1]['platforms'], crs=4326).reset_index().explore(m=m1, color='red')
m1

In [ ]:
t = [(1, 6), (0, 3), (2, 5)]
for i in range(3):


In [ ]:
test = pd.DataFrame(data_osm.iloc[2]['members'])
test = test[(test['type'] == 'node') & (test['role'].isin(['platform_entry_only', 'platform', 'platform_exit_only']))]
test['geometry'] = test.apply(lambda x: (Point(x["lon"], x["lat"])), axis=1)
test.reset_index(drop=True, inplace=True)
gdf = gpd.GeoDataFrame(test.reset_index(), crs=4326)
gdf.geometry = gdf.geometry.buffer(0.0001)
gdf.explore()

In [ ]:
test = pd.DataFrame(data_osm.iloc[1]['members'])
test = test[test['type'] == 'way']
test = test["geometry"]
test = test.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x]))
# gdf = gpd.GeoDataFrame(test.reset_index(),geometry='geometry',crs=4326)
# gdf.geometry = gdf.geometry.buffer(0.0001)
# gdf.explore()
test

In [ ]:
test

#### ПОПЫТКА СОЕДИНЯТЬ ДОРОГИ 

In [ ]:
from scipy.spatial import cKDTree
from shapely import LineString, Point
import geopandas as gpd

PLATFORM_ROLES = []


def parse_overpass_route_response(loc: dict):
    if 'ref' in loc['tags'].keys():
        transport_name = loc['tags']['ref']
    else:
        transport_name = loc['tags']['name']
    route = pd.DataFrame(loc["members"])

    ways = route[route["type"] == "way"]

    if len(ways) > 0:
        ways = ways["geometry"].reset_index(drop=True)
        ways = ways.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x])).tolist()
        if ways[0][0] == ways[1][0] or ways[0][0] == ways[1][-1]:
            complete_line = ways[0][::-1]
        else:
            complete_line = ways[0]
        for coords in ways[1:]:
            if complete_line[-1] == coords[0]:
                complete_line += coords[1:]
            elif complete_line[-1] == coords[-1]:
                complete_line += coords[::-1][1:]
            else:
                if coords[0] == coords[-1]:
                    continue
                else:
                    tree = cKDTree([coords[0], coords[-1]])
                    _, ind = tree.query(complete_line[-1])
                    if ind == 1:
                        coords = coords[::-1]
                    complete_line += coords
        # print('^^^^^^^^^^')

    else:
        complete_line = None
    if "node" in route["type"].unique():
        platforms = route[(route["type"] == "node")].apply(lambda x: Point(x["lon"], x["lat"]), axis=1)
    else:
        platforms = None

    return pd.Series({"way": complete_line, "platforms": platforms, 'name': transport_name})


data_geom = data_osm.apply(parse_overpass_route_response, axis=1, result_type='expand')
data_geom

In [ ]:
test = [LineString(
    [(46.0046822, 51.5426408), (46.0046214, 51.5425402), (46.0042506, 51.541918), (46.0038134, 51.5411845),
     (46.0036437, 51.5408997), (46.0035859, 51.5408011), (46.0035352, 51.5407158), (46.0031053, 51.5399929),
     (46.0028598, 51.5395801), (46.0026303, 51.5391941), (46.0025706, 51.5390938), (46.0023997, 51.5391331),
     (46.00211, 51.5391985), (46.0019414, 51.5392372), (46.0008809, 51.5394906), (46.00081, 51.5395332),
     (46.000665, 51.5396234), (46.0006024, 51.5396705), (46.0005255, 51.5397869), (46.000459, 51.5399285),
     (46.0005332, 51.54006), (46.0007185, 51.5403886), (46.0008003, 51.5405164), (46.0008441, 51.5405848)]), LineString(
    [(46.0046822, 51.5426408), (46.0048226, 51.5426046), (46.0048853, 51.5425884), (46.0065015, 51.5422214),
     (46.0067008, 51.5421762), (46.007038, 51.5420996), (46.0071807, 51.5420672)]), Point(46.0046822, 51.5426408)]

gpd.GeoDataFrame(geometry=test, crs=4326).explore()

In [ ]:
gpd.GeoDataFrame(geometry=data_geom['way'].apply(LineString), crs=4326).reset_index().explore(column='index')

In [ ]:
print(data_geom['way'].apply(LineString).iloc[1])

In [ ]:
data_geom.iloc[16]['way']

In [ ]:
ways = data_geom.iloc[16]['way']
ways = ways["geometry"].reset_index(drop=True)
lines = ways.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x])).tolist()
if lines[0][0] == lines[1][0]:
    complete_line = lines[0][::-1]
else:
    complete_line = lines[0]
for coords in lines[1:]:
    if complete_line[-1] == coords[0]:
        complete_line += coords[1:]
    elif complete_line[-1] == coords[-1]:
        complete_line += coords[::-1][1:]
    else:
        print('\n\nWTF\n\n')



In [ ]:
gpd.GeoDataFrame(geometry=[LineString(complete_line)], crs=4326).explore()

In [ ]:

from shapely import MultiPoint

gpd.GeoDataFrame(data=[{'kek': i} for i in data_geom.index.tolist()],
                 geometry=data_geom['platforms'].apply(lambda x: MultiPoint(x)), crs=4326).explore(column='kek')

In [ ]:

data_geom.index.tolist()

## Download from microsoft ML footprint

"""
This snippet demonstrates how to access and convert the buildings
data from .csv.gz to geojson for use in common GIS tools. You will
need to install pandas, geopandas, and shapely.
"""

import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

location = 'Russia'

dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
greece_links = dataset_links[dataset_links.Location == location]


def convert_to_bytes(size):
    size = str(size)
    if 'KB' in size:
        return float(size.replace('KB', '')) * 1024
    elif 'MB' in size:
        return float(size.replace('MB', '')) * 1024 * 1024
    elif 'B' in size:
        return float(size.replace('B', ''))
    else:
        return float(size)


greece_links['Size_in_bytes'] = greece_links['Size'].apply(convert_to_bytes)
greece_links = greece_links.sort_values(by='Size_in_bytes', ascending=False)
greece_links = greece_links.head(10)

greece_links = greece_links
for _, row in greece_links.iterrows():
    df = pd.read_json(row.Url, lines=True)
    df['geometry'] = df['geometry'].apply(shape)
    gdf = gpd.GeoDataFrame(df, crs=4326)
    gdf.to_file(f"{row.QuadKey}.geojson", driver="GeoJSON")



location = 'Russia'

dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
greece_links = dataset_links[dataset_links.Location == location]


def convert_to_bytes(size):
    size = str(size)
    if 'KB' in size:
        return float(size.replace('KB', '')) * 1024
    elif 'MB' in size:
        return float(size.replace('MB', '')) * 1024 * 1024
    elif 'B' in size:
        return float(size.replace('B', ''))
    else:
        return float(size)


greece_links['Size_in_bytes'] = greece_links['Size'].apply(convert_to_bytes)
greece_links = greece_links.sort_values(by='Size_in_bytes', ascending=False)

## Для вшэ временно тут
import math

import geopandas as gpd
f1 = 'Зоны видимости школ (без деревьев).geojson'
f2 = 'Зоны видимости школ (с деревьями).geojson'

d1 = gpd.read_file(f1).to_crs(32636)
d2 = gpd.read_file(f2).to_crs(32636)

d1.drop(columns=['area','perimeter'],inplace=True)
d2.drop(columns=['area','perimeter'],inplace=True)

import pandas as pd
schools = gpd.read_file('Полигоны школ.geojson').to_crs(32636)
schools['ID школы'] = schools['ID школы'].astype(int)
schools['Площадь школы(м2)'] = round(schools.geometry.area,2)
schools['school_geom'] = schools.geometry
d1_ =  pd.merge(d1,schools[['Площадь школы(м2)','ID школы','school_geom']],on='ID школы')
d2_ =  pd.merge(d2,schools[['Площадь школы(м2)','ID школы','school_geom']],on='ID школы')

import math

d1_['Площадь видимости(м2)'] = round(d1_.geometry.area,2)
d2_['Площадь видимости(м2)'] = round(d2_.geometry.area,2)

d1_['Коэффициент видимости'] = d1_.apply(lambda x: round(x.geometry.area / x['school_geom'].buffer(1000).area,5),axis=1)
d2_['Коэффициент видимости'] = d2_.apply(lambda x: round(x.geometry.area / x['school_geom'].buffer(1000).area,5),axis=1)

d1_.drop(columns='school_geom').to_file(f1)
d2_.drop(columns='school_geom').to_file(f2)